<h1> Instalação das bibliotecas necessárias </h1>

In [48]:
!pip install --upgrade pip
!pip install pymongo
!pip install -U nltk
!pip install joblib
!pip install pandas
!pip install sklearn

Requirement already up-to-date: pip in /opt/conda/lib/python3.7/site-packages (19.1.1)
Requirement already up-to-date: nltk in /opt/conda/lib/python3.7/site-packages (3.4.3)


In [1]:
from pymongo import MongoClient
import pandas as pd
import re
import nltk

from nltk.tokenize import TreebankWordTokenizer
from nltk.corpus import stopwords
from nltk.stem import SnowballStemmer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn import svm
from sklearn import tree
from sklearn.linear_model import LogisticRegression
from joblib import dump, load
from sklearn.model_selection import cross_val_score

#Download do corpus da nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/jovyan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

<h3>Definição das funções de leitura dos dados no MongoDb</h3>

In [2]:
def _connect_mongo(host, port, username, password, db):
    """ A util for making a connection to mongo """

    if username and password:
        mongo_uri = 'mongodb://%s:%s@%s:%s/%s' % (username, password, host, port, db)
        conn = MongoClient(mongo_uri)
    else:
        conn = MongoClient(host, port)


    return conn[db]

In [3]:
def read_mongo(db, collection, query={}, host='ds249824.mlab.com', port='49824', username='app', password='nodeapp01', no_id=True):
    """ Read from Mongo and Store into DataFrame """

    # Connect to MongoDB
    db = _connect_mongo(host=host, port=port, username=username, password=password, db=db)

    # Make a query to the specific DB and Collection
    cursor = db[collection].find(query)

    # Expand the cursor and construct the DataFrame
    df =  pd.DataFrame(list(cursor))

    # Delete the _id
    if no_id:
        del df['_id']

    return df

<h3>Carrega os DataSets (tweets marcados como acidente e não acidente)</h3>
<p> Cria Dataset chamado "acidente", com os documentos rotulados como Sim.
<p> Cria Dataset chamado "nao acidente", com os documentos rotulados como Não.
<p> Cria Dataset chamado "full_data", com os documentos rotulados como todos.

In [4]:
acidente = read_mongo('labeling_zone','tweets',query={'label':'Sim'})
nao_acidente = read_mongo('labeling_zone','tweets',query={'label':'Não'})
full_data = acidente.append(nao_acidente[0:277])
#full_data = acidente.append(nao_acidente)

In [5]:
print('Total de documentos rotulados como "acidente": ' + str(len(acidente)))
print('Total de documentos rotulados como "Não acidente": ' + str(len(nao_acidente)))

full_data[['lang','location','texto','usuario','label']].head()

Total de documentos rotulados como "acidente": 277
Total de documentos rotulados como "Não acidente": 1506


,lang,location,texto,usuario,label
0,pt,São Paulo,As faixas foram liberadas na Rod Dom Pedro I (...,radiotransitofm,Sim
1,pt,"São Paulo/SP, Brasil",Um dos caminhões e a carreta tombaram no acide...,TransitoSampaSP,Sim
2,pt,São Paulo,Ouvintes relatam #acidente no km 39 da rodovia...,radiotransitofm,Sim
3,pt,São Paulo/SP - Brasil,"13h24 - Acidente de trânsito, carro x moto, na...",BombeirosPMESP,Sim
4,pt,"São Paulo/SP, Brasil","Acidente de trânsito, carro x moto, na R.Pedro...",TransitoSampaSP,Sim


<h2>Aplica as funções de Pré-Processamento</h2>
<ul> 
    <li>Remoção de Urls (regex)</li>
    <li>Conversão para minúsculo</li>
    <li>Tokenização</li>
    <li>Remoção de Stop Words</li>
    <li>Steming</li>    
</ul>

In [6]:
def remove_urls(text):
    return re.sub(r'https?:\/\/.*[\r\n]*', '', text)

def to_lower(text):
    return text.lower()

<p>Remoção de URLs</p>
<p>Conversão para Minúsculo</p>

In [7]:
full_data['texto_formatado'] = full_data['texto'].apply(lambda t: remove_urls(str(t)))
full_data['texto_formatado'] = full_data['texto_formatado'].apply(lambda t: to_lower(t))

<p> Tokenização (Conversão de frases em array de tokens)

In [8]:
pd.options.display.max_colwidth = 100
tokenizer = TreebankWordTokenizer()
full_data['tokens'] = full_data['texto_formatado'].apply(lambda t: tokenizer.tokenize(t))
full_data[['texto_formatado','tokens']].head(6)

,texto_formatado,tokens
0,"as faixas foram liberadas na rod dom pedro i (sp-065) no km 33, em igaratá, região do vale do pa...","[as, faixas, foram, liberadas, na, rod, dom, pedro, i, (, sp-065, ), no, km, 33, ,, em, igaratá,..."
1,"um dos caminhões e a carreta tombaram no acidente, deixando cair na pista as cargas de peças de aç","[um, dos, caminhões, e, a, carreta, tombaram, no, acidente, ,, deixando, cair, na, pista, as, ca..."
2,"ouvintes relatam #acidente no km 39 da rodovia dos bandeirantes, sentido sp. lentidão de 1 km.","[ouvintes, relatam, #, acidente, no, km, 39, da, rodovia, dos, bandeirantes, ,, sentido, sp., le..."
3,"13h24 - acidente de trânsito, carro x moto, na rua pedro de toledo, 1651 – vila mariana. uma vít...","[13h24, -, acidente, de, trânsito, ,, carro, x, moto, ,, na, rua, pedro, de, toledo, ,, 1651, –,..."
4,"acidente de trânsito, carro x moto, na r.pedro de toledo, 1651 – vila mariana. uma vítima, masc.,","[acidente, de, trânsito, ,, carro, x, moto, ,, na, r.pedro, de, toledo, ,, 1651, –, vila, marian..."
5,"bloqueio total do acesso do #rodoanel no sentido perus, pista interna, para a #imigrantes na dir...","[bloqueio, total, do, acesso, do, #, rodoanel, no, sentido, perus, ,, pista, interna, ,, para, a..."


In [9]:
def create_stopword_list():
    portuguese_stops = set(stopwords.words('portuguese'))
    portuguese_stops.add('rt')

    with open('punctuation.txt','r+') as punct_file:
        puncts = punct_file.readlines()

    for item in puncts:    
        portuguese_stops.add(item.strip())
        
    return portuguese_stops

stop_w = create_stopword_list()

#aplica a remocao de stop-words
full_data['words'] = full_data['tokens'].apply(lambda w: [word for word in w if word not in stop_w]) 
#exibe resultado intermediario
full_data[['tokens','words']].head(10)

,tokens,words
0,"[as, faixas, foram, liberadas, na, rod, dom, pedro, i, (, sp-065, ), no, km, 33, ,, em, igaratá,...","[faixas, liberadas, rod, dom, pedro, i, sp-065, km, 33, igaratá, região, vale, paraíba, após, gr..."
1,"[um, dos, caminhões, e, a, carreta, tombaram, no, acidente, ,, deixando, cair, na, pista, as, ca...","[caminhões, carreta, tombaram, acidente, deixando, cair, pista, cargas, peças, aç]"
2,"[ouvintes, relatam, #, acidente, no, km, 39, da, rodovia, dos, bandeirantes, ,, sentido, sp., le...","[ouvintes, relatam, acidente, km, 39, rodovia, bandeirantes, sentido, sp., lentidão, 1, km]"
3,"[13h24, -, acidente, de, trânsito, ,, carro, x, moto, ,, na, rua, pedro, de, toledo, ,, 1651, –,...","[13h24, acidente, trânsito, carro, x, moto, rua, pedro, toledo, 1651, –, vila, mariana., vítima,..."
4,"[acidente, de, trânsito, ,, carro, x, moto, ,, na, r.pedro, de, toledo, ,, 1651, –, vila, marian...","[acidente, trânsito, carro, x, moto, r.pedro, toledo, 1651, –, vila, mariana., vítima, masc.]"
5,"[bloqueio, total, do, acesso, do, #, rodoanel, no, sentido, perus, ,, pista, interna, ,, para, a...","[bloqueio, total, acesso, rodoanel, sentido, perus, pista, interna, imigrantes, direção, litoral..."
6,"[ouvintes, relatam, #, acidente, no, km, 39, da, rodovia, dos, bandeirantes, ,, sent, sp., lenti...","[ouvintes, relatam, acidente, km, 39, rodovia, bandeirantes, sent, sp., lentidão, 1, km, radiotr..."
7,"[capotamento, no, #, corredornortesul, sentido, santana, ,, na, altura, de, congonhas., ouvintes...","[capotamento, corredornortesul, sentido, santana, altura, congonhas., ouvintes, relatam, capotam..."
8,"[capotamento, no, #, corredornortesul, sentido, santana, ,, na, altura, de, congonhas., ouvintes...","[capotamento, corredornortesul, sentido, santana, altura, congonhas., ouvintes, relatam, capotam..."
9,"[capotamento, no, #, corredornortesul, sentido, santana, ,, na, altura, de, congonhas., ouvintes...","[capotamento, corredornortesul, sentido, santana, altura, congonhas., ouvintes, relatam, capotam..."


In [10]:
stemer= SnowballStemmer(language='portuguese')
full_data['stem_words'] = full_data['words'].apply(lambda t: [stemer.stem(word) for word in t])
full_data[['words','stem_words']].head()

,words,stem_words
0,"[faixas, liberadas, rod, dom, pedro, i, sp-065, km, 33, igaratá, região, vale, paraíba, após, gr...","[faix, liber, rod, dom, pedr, i, sp-065, km, 33, igarat, regiã, val, paraíb, após, grav, acident..."
1,"[caminhões, carreta, tombaram, acidente, deixando, cair, pista, cargas, peças, aç]","[caminhõ, carret, tomb, acident, deix, cair, pist, carg, pec, ac]"
2,"[ouvintes, relatam, acidente, km, 39, rodovia, bandeirantes, sentido, sp., lentidão, 1, km]","[ouvint, relat, acident, km, 39, rodov, bandeir, sent, sp., lentidã, 1, km]"
3,"[13h24, acidente, trânsito, carro, x, moto, rua, pedro, toledo, 1651, –, vila, mariana., vítima,...","[13h24, acident, trânsit, carr, x, mot, rua, pedr, toled, 1651, –, vil, mariana., vítim, masc., ..."
4,"[acidente, trânsito, carro, x, moto, r.pedro, toledo, 1651, –, vila, mariana., vítima, masc.]","[acident, trânsit, carr, x, mot, r.pedr, toled, 1651, –, vil, mariana., vítim, masc.]"


In [11]:
def encode_label(text):
    if text=='Sim':
        return 1
    elif text=='Não':
        return 0
    else:
        return -1

full_data['target'] = full_data['label'].apply(lambda label: encode_label(label))
full_data['target'].describe()

count    554.000000
mean       0.500000
std        0.500452
min        0.000000
25%        0.000000
50%        0.500000
75%        1.000000
max        1.000000
Name: target, dtype: float64

In [12]:
full_data['clean_text'] = full_data['stem_words'].apply(lambda t: str(' '.join(t)))

count_vect = CountVectorizer()
X_TF = count_vect.fit_transform(full_data['clean_text'])
print(X_TF.shape)

(554, 1991)


In [13]:
print(X_TF.toarray()) 

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]]


In [17]:
tfidf_transformer = TfidfTransformer()
X = tfidf_transformer.fit_transform(X_TF)
X.shape

(554, 1991)

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X, full_data['target'], test_size=0.33, random_state=42)

In [19]:
nb = MultinomialNB().fit(X_train, y_train)
svm = svm.SVC().fit(X_train,y_train)
tree = tree.DecisionTreeClassifier().fit(X_train,y_train)
logr = LogisticRegression().fit(X_train,y_train)

/opt/conda/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/opt/conda/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [101]:
nbc = MultinomialNB()
svmc = svm.SVC()
treec = tree.DecisionTreeClassifier()
logr = LogisticRegression()

AttributeError: 'SVC' object has no attribute 'SVC'

<h1> Realiza o cross validation </h1>

In [ ]:
nb_scores = cross_val_score(nbc, X, full_data['target'], cv=5)
svm_scores = cross_val_score(svmc, X, full_data['target'], cv=5)
tree_scores = cross_val_score(treec, X, full_data['target'], cv=5)
logr_scores = cross_val_score(logr, X, full_data['target'], cv=5)

In [45]:
print(nb_scores)
print(svm_scores)
print(tree_scores)
print(logr_scores)

[0.87430168 0.84593838 0.85393258 0.87078652 0.86235955]
[0.84357542 0.84313725 0.84550562 0.84550562 0.84550562]
[0.87988827 0.85994398 0.89325843 0.89044944 0.91011236]
[0.87150838 0.86554622 0.85955056 0.87921348 0.86797753]


In [20]:
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

def print_metrics(y_true,y_predicted):
    acc = accuracy_score(y_test.values,y_predicted)
    print('Acurácia: ' + str(acc))
    #accuracy_score(y_true, y_predicted)
    print('Matriz de Confusão:' )
    print(confusion_matrix(y_true, y_predicted, labels=[0, 1]))
    print('Recall')
    print(recall_score(y_true, y_predicted, average='macro')  )
    print('Precision')
    print(precision_score(y_true, y_predicted, average='macro')  )

In [21]:
print('Naive Bayes')
print_metrics(y_test,nb.predict(X_test))
print('\n')
print('Decision Tree')
print_metrics(y_test,tree.predict(X_test))
print('\n')
print('Suport Vector Machines')
print_metrics(y_test,svm.predict(X_test))
print('\n')
print('Logistic Regressor')
print_metrics(y_test,logr.predict(X_test))

Naive Bayes
Acurácia: 0.825136612021858
Matriz de Confusão:
[[73 19]
 [13 78]]
Recall
0.8253105590062111
Precision
0.8264804603212659


Decision Tree
Acurácia: 0.8852459016393442
Matriz de Confusão:
[[80 12]
 [ 9 82]]
Recall
0.8853320592451027
Precision
0.8856084150131485


Suport Vector Machines
Acurácia: 0.4972677595628415
Matriz de Confusão:
[[ 0 92]
 [ 0 91]]
Recall
0.5
Precision
0.24863387978142076


Logistic Regressor
Acurácia: 0.907103825136612
Matriz de Confusão:
[[81 11]
 [ 6 85]]
Recall
0.9072503583373148
Precision
0.9082255747126436


/opt/conda/lib/python3.7/site-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
